**ASSIGNMENT 1 - EMPIRICAL STUDY OF KNAPSACK PROBLEM**

**1. Group Description**

Group Number: 100 \
Member Names: Sébastien Girard and Zachary Legros \
Member Student Numbers:
- Sébastien Girard: 300133000
- Zachary Legros: 300136274

**2. Knapsack Problem**

Give a description of the problem tackled.

**3. Dataset**

Give a description of the dataset used with references.  

**Import important libraries**

In [83]:
import pandas as pd
import itertools
import numpy as np

**Read Dataset**

As outlined in the project description, it should be possible for the correctors to execute your notebook without requiring any downloads.

To facilitate access to the dataset without the need for downloads, you can upload it to a public GitHub repository and provide a link to the raw version of the dataset.

The link to the raw version is as follows:
*https://raw.githubusercontent.com/GITHUB_USERNAME/REPOSITORY_NAME/main/DATASETNAME.csv*

For example:

https://raw.githubusercontent.com/baharin/KnapsackProblem/main/knapsack_5_items.csv

Now provide the link to YOUR dataset and read the dataset using pandas:

In [84]:
url = "https://raw.githubusercontent.com/sebastiengrd/AI-Assignment1/main/knapsack_5_items.csv"

dataset = pd.read_csv(url)

Let's see what are the columns of the dataset? :

In [85]:
dataset.columns

Index(['Weights', 'Prices', 'Capacity', 'Best picks', 'Best price'], dtype='object')

As we expected, we have columns for weights, costs, capacity, best picks and best price for all the instances.

Now let's see the first 10 entries (rows):

In [86]:
dataset.head(10)

,Weights,Prices,Capacity,Best picks,Best price
0,[46 40 42 38 10],[12 19 19 15 8],40,[0. 1. 0. 0. 0.],19.0
1,[11 31 4 6 7],[ 2 8 18 16 3],64,[1. 1. 1. 1. 1.],47.0
2,[32 49 27 37 24],[19 16 16 4 1],87,[1. 0. 1. 0. 1.],36.0
3,[20 35 22 23 16],[19 17 19 9 1],21,[1. 0. 0. 0. 0.],19.0
4,[ 7 12 19 13 20],[10 11 18 15 5],50,[0. 1. 1. 1. 0.],44.0
5,[27 10 25 25 7],[13 19 7 16 3],66,[1. 1. 0. 1. 0.],48.0
6,[21 2 33 45 26],[ 1 14 10 6 13],80,[0. 1. 1. 0. 1.],37.0
7,[37 27 39 14 25],[18 7 15 4 13],35,[0. 0. 0. 0. 1.],13.0
8,[ 1 48 4 23 39],[ 9 4 10 16 12],51,[1. 0. 1. 1. 0.],35.0
9,[ 4 3 22 9 32],[14 6 3 17 8],53,[1. 1. 0. 1. 1.],45.0


**Preprocessing Step**

Typically, the initial step in any project that involves reading and handling data is data preprocessing and cleansing.

In our dataset, we expect the entries in the "Weights," "Prices," and "Best Picks" columns to be in the form of arrays of floats or integers, like this: [45, 40, 42, 38, 10]

However, when you read each entry using pandas, they will be in a form of a string: "[45 40 42 38 10]"

So we need to convert these strings into "arrays of floats or integers." You can utilize the function provided below for this purpose:


In [87]:
def string_to_list(string):

  string_list = string.strip('[]').split()

  float_list = [float(element) for element in string_list]

  return float_list

Furthermore, it's possible that certain rows in the dataset contain empty values in specific columns. We also aim to eliminate these rows as they do not provide any useful information. We use dropna() function to do so:

In [88]:
#Ignore the warning messages.

dataset = dataset.dropna()

dataset.Weights = dataset.Weights.apply(lambda x : string_to_list(x))
dataset.Prices = dataset.Prices.apply(lambda x : string_to_list(x))
dataset['Best picks'] = dataset['Best picks'].apply(lambda x : string_to_list(x))

Now it's time to implement the search algorithms. For each algorithm, a template is provided to you. You can modify this template if you want. But first you should try to go look at all the parameters used, as they are all important. You can also define any number of auxiliary functions you want.


**4. Generate and Test**

Give a description... Also, make sure that you put comments using your own words in the code to show that you understand the code that you are submitting.

In [89]:
solutions = []
# for the first 5 rows in the dataset
for _, row in dataset.iterrows():
    target = row['Best price']
    print(type(row.Weights))
    break
    

<class 'list'>


In [90]:

# recursive way to solve the {0 .. 1} knapsack problem
# this function takes the weight, the values, and the weights of the items
# it evaluates two scenarios, one where it takes the last item, and one where it doesn't
# then, it returns the maximum value of the two scenarios, with the two items selected
def knapsack_solver(weight, values, weights):
  
  # base case. If there is no values left, or the weight is 0, then we return 0
  if (weight == 0 or len(values) == 0):
    return 0, []
  
  do_not_take_last, do_not_take_last_result = knapsack_solver(weight, values[:-1], weights[:-1])
  take_last, take_last_result = knapsack_solver(weight - weights[-1], values[:-1], weights[:-1])

  if (take_last > do_not_take_last):
    return take_last + values[-1], take_last_result + [1]
  
  return do_not_take_last, do_not_take_last_result + [0]
  

def gen_and_test(data): #takes one row, and must compute the best solution for that row
  # implement knapsack solution
  

  best_solution_price, best_solution = knapsack_solver(data['Capacity'], data['Prices'], data['Weights'])

  return best_solution_price, best_solution



In [91]:
solutions = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = gen_and_test(row)
    print(solution, indexes, target)
    solutions.append(1 if target == solution else 0)
    break


0 [0, 0, 0, 0, 0] 19.0


In [92]:
# Accuracy
print('Accuracy of best prices found is', np.mean(solutions))

Accuracy of best prices found is 0.0


**Your Analysis:**

------------------------------------------------------------------------------------------------

**5. Greedy Search**

Give a description... Also, make sure that you put comments using your own words in the code to show that you understand the code that you are submitting.

In [93]:
def greedy(data):

  return best_solution_price, best_solution


In [94]:
solutions_greedy = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = greedy(row)
    solutions_greedy.append(1 if target == solution else 0)


NameError: name 'best_solution_price' is not defined

In [ ]:
print("Greedy Accuracy is", np.mean(solutions_greedy))

**Your Analysis:**

------------------------------------------------------------------------------------------------

**6. Simulated Annealing**

Give a description... Also, make sure that you put comments using your own words in the code to show that you understand the code that you are submitting.


In [429]:
import random
import math


def sum_of_product(arr1, arr2):
  return (np.array(arr1) * arr2).sum()


def cost(x, data):
  return data['Capacity'] - sum_of_product(data['Prices'], x)

# Accept the candidate solution if it has a higher energy (value) and
# a weight that is smaller than the capacity
def accept(candidate_picks, delta_e, t, data):
  total_weight = sum_of_product(data['Weights'], candidate_picks)
  if total_weight > data['Capacity']:
    return False
  else:
    try:
      prob = 1 / (1 + math.exp(delta_e/t))
      r = random.random()
      return r < prob
    except OverflowError:
      return False
    


def generate_candidate():
  return [round(random.random()) for _ in range(5)]


def flip_random(x):
  new_x = x.copy()
  r_i = round(random.uniform(0, len(new_x) - 1))
  new_x[r_i] = int(not new_x[r_i])
  return new_x


def simulated_annealing(data, N, initial_temperature, cooling_rate):
  t = initial_temperature
  x = [0,0,0,0,0]
  e = cost(x, data)
  it = 0

  while it < N:
    candidate = flip_random(x)
    candidate_e = cost(candidate, data)
    delta_e = candidate_e - e
    if accept(candidate, delta_e, t, data):
      x = candidate
      e = candidate_e
    it += 1
    t *= cooling_rate
  
  return sum_of_product(data['Prices'], x), x

test_row = {
  "Weights": [46, 40, 42, 38, 10],
  "Prices": [12, 19, 19, 15,  8],
  "Capacity": 40,
  "Best picks": [0., 1., 0., 0., 0.],
  "Best price": 19.0
}

solution, indexes = simulated_annealing(test_row, N = 1000, initial_temperature=1, cooling_rate=0.95)
print(solution, indexes)

15 [0, 0, 0, 1, 0]


In [412]:
solutions_sa = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = simulated_annealing(row, N = 1000, initial_temperature=1, cooling_rate=0.95)
    solutions_sa.append(1 if target == solution else 0)


KeyboardInterrupt: 

In [411]:
print("Simulated Annealing Accuracy is", np.mean(solutions_sa))

Simulated Annealing Accuracy is 0.49104160340115394


**Your Analysis:**

------------------------------------------------------------------------------------------------

**7. Genetic Algorithm**

Give a description... Also, make sure that you put comments using your own words in the code to show that you understand the code that you are submitting.

In [ ]:
import random

def calculate_fitness(ind, prices, weights, capacity):


  return fitness

def crossover(parent1, parent2, cross_rate):


  return child1, child2

def mutation(child, mut_rate):


  return child

def genetic_algorithm(data, population_size, num_generations, mut_rate, cross_rate, tournament_size):


  return best_solution_price, best_solution

**Your Analysis:**

------------------------------------------------------------------------------------------------

**8. Comparative Study**

description  +  tables/figures

--------------------------------------------------------------------------


**9. Conclusion**

Comment on the empirical study, its results, and give ideas for future work.

--------------------------------------------------------------------------


**10 References**

Make sure you provide references to ALL sources used (articles, code, algorithms).

**Hint:** To share a link to your colab notebook, click on "share" on the top right. Then, under *General access* , change *Restricted* to "Anyone with the link".